In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import os
from citipy import citipy

In [3]:
csv_path = "RawData/Zip_Zhvi_SingleFamilyResidence.csv"

In [4]:
singlefam_data = pd.io.parsers.read_csv(csv_path, encoding='ISO-8859-1',dtype={'RegionName': 'str'})
singlefam_data['RegionName'] = singlefam_data['RegionName'].apply(lambda x: x.zfill(5))
singlefam_data.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,2016-01,2016-02,2016-03,...,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12
0,61639,10025,New York,NY,New York-Newark-Jersey City,New York County,1,1416633.0,1427321.0,1441979.0,...,1380332,1386270,1394397,1404225,1406599,1399918,1380178,1358401,1350481,1345845
1,84654,60657,Chicago,IL,Chicago-Naperville-Elgin,Cook County,2,907374.0,921322.0,930106.0,...,966123,967557,965155,960225,956709,953095,950684,948136,946838,945928
2,61637,10023,New York,NY,New York-Newark-Jersey City,New York County,3,1556960.0,1598874.0,1627446.0,...,1520816,1526676,1525174,1516721,1504180,1492086,1480934,1468075,1454870,1438313
3,91982,77494,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,4,336604.0,335648.0,334774.0,...,335789,335035,334542,334176,334363,334127,334458,334460,334679,334309
4,84616,60614,Chicago,IL,Chicago-Naperville-Elgin,Cook County,5,1114867.0,1131654.0,1142148.0,...,1198898,1203423,1204840,1199747,1194591,1188702,1183500,1179024,1175407,1174008


In [5]:
singlefam_data.columns.values.tolist()

['RegionID',
 'RegionName',
 'City',
 'State',
 'Metro',
 'CountyName',
 'SizeRank',
 '2016-01',
 '2016-02',
 '2016-03',
 '2016-04',
 '2016-05',
 '2016-06',
 '2016-07',
 '2016-08',
 '2016-09',
 '2016-10',
 '2016-11',
 '2016-12',
 '2017-01',
 '2017-02',
 '2017-03',
 '2017-04',
 '2017-05',
 '2017-06',
 '2017-07',
 '2017-08',
 '2017-09',
 '2017-10',
 '2017-11',
 '2017-12',
 '2018-01',
 '2018-02',
 '2018-03',
 '2018-04',
 '2018-05',
 '2018-06',
 '2018-07',
 '2018-08',
 '2018-09',
 '2018-10',
 '2018-11',
 '2018-12',
 '2019-01',
 '2019-02',
 '2019-03',
 '2019-04',
 '2019-05',
 '2019-06',
 '2019-07',
 '2019-08',
 '2019-09',
 '2019-10',
 '2019-11',
 '2019-12']

In [6]:
#Drop unnecessary columns, will analyze 2018 data
zillow_2016_data = singlefam_data.drop([
 '2017-01',
 '2017-02',
 '2017-03',
 '2017-04',
 '2017-05',
 '2017-06',
 '2017-07',
 '2017-08',
 '2017-09',
 '2017-10',
 '2017-11',
 '2017-12',
 '2018-01',
 '2018-02',
 '2018-03',
 '2018-04',
 '2018-05',
 '2018-06',
 '2018-07',
 '2018-08',
 '2018-09',
 '2018-10',
 '2018-11',
 '2018-12',
 '2019-01',
 '2019-02',
 '2019-03',
 '2019-04',
 '2019-05',
 '2019-06',
 '2019-07',
 '2019-08',
 '2019-09',
 '2019-10',
 '2019-11',
 '2019-12'], axis=1)
zillow_2016_data.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12
0,61639,10025,New York,NY,New York-Newark-Jersey City,New York County,1,1416633.0,1427321.0,1441979.0,1458433.0,1470286.0,1475276.0,1480967.0,1483113.0,1486790.0,1490909.0,1493113.0,1500163.0
1,84654,60657,Chicago,IL,Chicago-Naperville-Elgin,Cook County,2,907374.0,921322.0,930106.0,932259.0,929455.0,927464.0,929221.0,928802.0,926166.0,926066.0,929987.0,931155.0
2,61637,10023,New York,NY,New York-Newark-Jersey City,New York County,3,1556960.0,1598874.0,1627446.0,1646238.0,1648779.0,1646252.0,1635584.0,1630020.0,1627167.0,1630578.0,1630922.0,1629389.0
3,91982,77494,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,4,336604.0,335648.0,334774.0,334665.0,335081.0,333881.0,332303.0,330534.0,329476.0,328831.0,328912.0,329142.0
4,84616,60614,Chicago,IL,Chicago-Naperville-Elgin,Cook County,5,1114867.0,1131654.0,1142148.0,1150378.0,1149627.0,1148597.0,1147756.0,1145955.0,1141840.0,1140892.0,1144867.0,1147812.0


In [7]:
zillow_2016_data.dtypes

RegionID        int64
RegionName     object
City           object
State          object
Metro          object
CountyName     object
SizeRank        int64
2016-01       float64
2016-02       float64
2016-03       float64
2016-04       float64
2016-05       float64
2016-06       float64
2016-07       float64
2016-08       float64
2016-09       float64
2016-10       float64
2016-11       float64
2016-12       float64
dtype: object

In [8]:
zillow_2016_data['2016_AvgHomeValue'] = zillow_2016_data[['2016-01','2016-02','2016-03','2016-04','2016-05','2016-06',
                                                    '2016-07','2016-08','2016-09','2016-10',
                                                    '2016-11','2016-12']].sum(axis=1)/12

zillow_2016_data['2016_AvgHomeValue'] = zillow_2016_data['2016_AvgHomeValue'].astype(np.int32)
zillow_2016_data.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2016_AvgHomeValue
0,61639,10025,New York,NY,New York-Newark-Jersey City,New York County,1,1416633.0,1427321.0,1441979.0,1458433.0,1470286.0,1475276.0,1480967.0,1483113.0,1486790.0,1490909.0,1493113.0,1500163.0,1468748
1,84654,60657,Chicago,IL,Chicago-Naperville-Elgin,Cook County,2,907374.0,921322.0,930106.0,932259.0,929455.0,927464.0,929221.0,928802.0,926166.0,926066.0,929987.0,931155.0,926614
2,61637,10023,New York,NY,New York-Newark-Jersey City,New York County,3,1556960.0,1598874.0,1627446.0,1646238.0,1648779.0,1646252.0,1635584.0,1630020.0,1627167.0,1630578.0,1630922.0,1629389.0,1625684
3,91982,77494,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,4,336604.0,335648.0,334774.0,334665.0,335081.0,333881.0,332303.0,330534.0,329476.0,328831.0,328912.0,329142.0,332487
4,84616,60614,Chicago,IL,Chicago-Naperville-Elgin,Cook County,5,1114867.0,1131654.0,1142148.0,1150378.0,1149627.0,1148597.0,1147756.0,1145955.0,1141840.0,1140892.0,1144867.0,1147812.0,1142199


In [9]:
avg_2016_valdf=zillow_2016_data.drop(['2016-01','2016-02','2016-03','2016-04','2016-05','2016-06',
                       '2016-07','2016-08','2016-09','2016-10','2016-11',
                                      '2016-12', 'SizeRank'], axis=1).rename(columns={"RegionName": "ZipCode"})
avg_2016_valdf.head()

,RegionID,ZipCode,City,State,Metro,CountyName,2016_AvgHomeValue
0,61639,10025,New York,NY,New York-Newark-Jersey City,New York County,1468748
1,84654,60657,Chicago,IL,Chicago-Naperville-Elgin,Cook County,926614
2,61637,10023,New York,NY,New York-Newark-Jersey City,New York County,1625684
3,91982,77494,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,332487
4,84616,60614,Chicago,IL,Chicago-Naperville-Elgin,Cook County,1142199


In [10]:
#zip_group_avg = avg_2019_valdf.groupby(['ZipCode'])

In [11]:
zip_group_avg = avg_2016_valdf.groupby(['ZipCode','City','State','Metro','CountyName'])['2016_AvgHomeValue'].mean().reset_index()
zip_group_avg.head()


,ZipCode,City,State,Metro,CountyName,2016_AvgHomeValue
0,00501,Holtsville,NY,New York-Newark-Jersey City,Suffolk County,88583
1,00544,Simsboro,LA,Ruston,Lincoln Parish,80995
2,00602,Auburn,AL,Auburn-Opelika,Lee County,64707
3,00603,Aguadilla,AL,Dothan,Houston County,75064
4,00604,Aguadilla,AL,Dothan,Houston County,60068


In [12]:
zip_group_avg_sorted = zip_group_avg.sort_values(by='2016_AvgHomeValue', ascending=False).reset_index()
zip_group_avg_sorted.head()

,index,ZipCode,City,State,Metro,CountyName,2016_AvgHomeValue
0,2066,10065,New York,NY,New York-Newark-Jersey City,New York County,8959594
1,2068,10075,New York,NY,New York-Newark-Jersey City,New York County,8299805
2,2045,10021,New York,NY,New York-Newark-Jersey City,New York County,7677481
3,2039,10014,New York,NY,New York-Newark-Jersey City,New York County,6369047
4,2069,10128,New York,NY,New York-Newark-Jersey City,New York County,6354494


In [13]:
clean_2016avgprice_zip_df = zip_group_avg_sorted.drop(['index'],axis=1)
clean_2016avgprice_zip_df

,ZipCode,City,State,Metro,CountyName,2016_AvgHomeValue
0,10065,New York,NY,New York-Newark-Jersey City,New York County,8959594
1,10075,New York,NY,New York-Newark-Jersey City,New York County,8299805
2,10021,New York,NY,New York-Newark-Jersey City,New York County,7677481
3,10014,New York,NY,New York-Newark-Jersey City,New York County,6369047
4,10128,New York,NY,New York-Newark-Jersey City,New York County,6354494
5,81611,Aspen,CO,Glenwood Springs,Pitkin County,4666409
6,10013,New York,NY,New York-Newark-Jersey City,New York County,4656021
7,02108,Boston,MA,Boston-Cambridge-Newton,Suffolk County,4381094
8,94027,Atherton,CA,San Francisco-Oakland-Hayward,San Mateo County,4277024
9,11962,Sagaponack,NY,New York-Newark-Jersey City,Suffolk County,4037442


In [14]:
#export Dataframe to CSV - add to final_data directory
clean_2016avgprice_zip_df.to_csv("CleanData/Zillow_2016avghomeval.csv",index=False)